In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from xgboost import XGBRegressor
from datetime import date, datetime
from sklearn.impute import SimpleImputer
from IPython.core.display import display, HTML
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
%matplotlib inline
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))
print('Libraries imported successfully!!')

Libraries imported successfully!!


In [2]:
db = pd.read_csv('iplav.csv')
db.head()

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
0,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,1,0,0.1,1,0,0,0,222
1,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,0,0.2,1,0,0,0,222
2,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.2,2,0,0,0,222
3,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.3,2,0,0,0,222
4,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.4,2,0,0,0,222


In [3]:
db.drop(['mid', 'batsman', 'bowler', 'striker', 'non-striker'], axis = 1, inplace = True)

In [4]:
db.head()

,date,venue,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total
0,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.1,1,0,222
1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.2,1,0,222
2,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.2,2,0,222
3,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.3,2,0,222
4,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.4,2,0,222


In [5]:
db['yyyymmdd'] = db['date'].apply(lambda x : x.split()[0])
db['year'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').year)
db['month'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').month)
db['date'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').day)
db['day_number'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').weekday())
db = db.drop(['yyyymmdd'], axis = 1)

In [6]:
db.head()

,date,venue,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total,year,month,day_number
0,18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.1,1,0,222,2008,4,4
1,18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.2,1,0,222,2008,4,4
2,18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.2,2,0,222,2008,4,4
3,18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.3,2,0,222,2008,4,4
4,18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.4,2,0,222,2008,4,4


In [7]:
db.columns

Index(['date', 'venue', 'bat_team', 'bowl_team', 'runs', 'wickets', 'overs',
       'runs_last_5', 'wickets_last_5', 'total', 'year', 'month',
       'day_number'],
      dtype='object')

In [8]:
db.venue.unique()

array(['M Chinnaswamy Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Feroz Shah Kotla',
       'Wankhede Stadium', 'Eden Gardens', 'Sawai Mansingh Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'MA Chidambaram Stadium, Chepauk', 'Dr DY Patil Sports Academy',
       'Newlands', "St George's Park", 'Kingsmead', 'SuperSport Park',
       'Buffalo Park', 'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium',
       'Sardar Patel Stadium, Motera', 'Barabati Stadium',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium', 'Nehru Stadium',
       'Holkar Cricket Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sheikh Zayed Stadium',
       'Sharjah Cricket Stadium', 'Dubai International Cricket St

In [9]:
for col in ['bat_team', 'bowl_team']:
    t_list = []
    for i in db[col]:
        if i == 'Delhi Daredevils':
            t_list.append('Delhi Capitals')
        elif i == 'Rising Pune Supergiants':
            t_list.append('Rising Pune Supergiant')
        else:
            t_list.append(i)
    db[col] = t_list

In [10]:
df = db[db['year'] >= 2012]
df = df[df['overs'] >= 5.0]

In [11]:
df.head()

,date,venue,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total,year,month,day_number
29594,4,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Mumbai Indians,35,1,5.1,31,1,112,2012,4,2
29595,4,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Mumbai Indians,36,1,5.2,32,0,112,2012,4,2
29596,4,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Mumbai Indians,37,1,5.3,33,0,112,2012,4,2
29597,4,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Mumbai Indians,38,1,5.4,33,0,112,2012,4,2
29598,4,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Mumbai Indians,38,2,5.5,32,1,112,2012,4,2


In [12]:
dummy = ['venue', 'bat_team', 'bowl_team']
df_dummy = pd.get_dummies(df[dummy], prefix_sep = '_', drop_first = True)
df = df.drop(dummy, axis = 1)
df = pd.concat([df, df_dummy], axis = 1)

In [13]:
df.head()

,date,runs,wickets,overs,runs_last_5,wickets_last_5,total,year,month,day_number,venue_Brabourne Stadium,venue_Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium,venue_Dubai International Cricket Stadium,venue_Eden Gardens,venue_Feroz Shah Kotla,venue_Green Park,venue_Himachal Pradesh Cricket Association Stadium,venue_Holkar Cricket Stadium,venue_JSCA International Stadium Complex,venue_M Chinnaswamy Stadium,"venue_MA Chidambaram Stadium, Chepauk",venue_Maharashtra Cricket Association Stadium,"venue_Punjab Cricket Association Stadium, Mohali","venue_Rajiv Gandhi International Stadium, Uppal","venue_Sardar Patel Stadium, Motera",venue_Saurashtra Cricket Association Stadium,venue_Sawai Mansingh Stadium,venue_Shaheed Veer Narayan Singh International Stadium,venue_Sharjah Cricket Stadium,venue_Sheikh Zayed Stadium,venue_Subrata Roy Sahara Stadium,venue_Wankhede Stadium,bat_team_Deccan Chargers,bat_team_Delhi Capitals,bat_team_Gujarat Lions,bat_team_Kings XI Punjab,bat_team_Kolkata Knight Riders,bat_team_Mumbai Indians,bat_team_Pune Warriors,bat_team_Rajasthan Royals,bat_team_Rising Pune Supergiant,bat_team_Royal Challengers Bangalore,bat_team_Sunrisers Hyderabad,bowl_team_Deccan Chargers,bowl_team_Delhi Capitals,bowl_team_Gujarat Lions,bowl_team_Kings XI Punjab,bowl_team_Kolkata Knight Riders,bowl_team_Mumbai Indians,bowl_team_Pune Warriors,bowl_team_Rajasthan Royals,bowl_team_Rising Pune Supergiant,bowl_team_Royal Challengers Bangalore,bowl_team_Sunrisers Hyderabad
29594,4,35,1,5.1,31,1,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
29595,4,36,1,5.2,32,0,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
29596,4,37,1,5.3,33,0,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
29597,4,38,1,5.4,33,0,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
29598,4,38,2,5.5,32,1,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [14]:
df.shape

(34685, 54)

In [15]:
df.columns

Index(['date', 'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5',
       'total', 'year', 'month', 'day_number', 'venue_Brabourne Stadium',
       'venue_Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'venue_Dubai International Cricket Stadium', 'venue_Eden Gardens',
       'venue_Feroz Shah Kotla', 'venue_Green Park',
       'venue_Himachal Pradesh Cricket Association Stadium',
       'venue_Holkar Cricket Stadium',
       'venue_JSCA International Stadium Complex',
       'venue_M Chinnaswamy Stadium', 'venue_MA Chidambaram Stadium, Chepauk',
       'venue_Maharashtra Cricket Association Stadium',
       'venue_Punjab Cricket Association Stadium, Mohali',
       'venue_Rajiv Gandhi International Stadium, Uppal',
       'venue_Sardar Patel Stadium, Motera',
       'venue_Saurashtra Cricket Association Stadium',
       'venue_Sawai Mansingh Stadium',
       'venue_Shaheed Veer Narayan Singh International Stadium',
       'venue_Sharjah Cricket Stadium', 'venue_S

In [16]:
df = df[['runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5', 'total', 'year', 'bat_team_Deccan Chargers', 'bat_team_Delhi Capitals', 'bat_team_Gujarat Lions', 
         'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians', 'bat_team_Pune Warriors', 
         'bat_team_Rajasthan Royals', 'bat_team_Rising Pune Supergiant', 'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad', 
         'bowl_team_Deccan Chargers', 'bowl_team_Delhi Capitals', 'bowl_team_Gujarat Lions', 'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders', 
         'bowl_team_Mumbai Indians', 'bowl_team_Pune Warriors', 'bowl_team_Rajasthan Royals', 'bowl_team_Rising Pune Supergiant', 'bowl_team_Royal Challengers Bangalore', 
         'bowl_team_Sunrisers Hyderabad', 'venue_Brabourne Stadium', 'venue_Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', 
         'venue_Dubai International Cricket Stadium', 'venue_Eden Gardens', 'venue_Feroz Shah Kotla', 'venue_Green Park', 
         'venue_Himachal Pradesh Cricket Association Stadium', 'venue_Holkar Cricket Stadium', 'venue_JSCA International Stadium Complex', 
         'venue_M Chinnaswamy Stadium', 'venue_MA Chidambaram Stadium, Chepauk', 'venue_Maharashtra Cricket Association Stadium', 
         'venue_Punjab Cricket Association Stadium, Mohali', 'venue_Rajiv Gandhi International Stadium, Uppal', 'venue_Sardar Patel Stadium, Motera', 
         'venue_Saurashtra Cricket Association Stadium', 'venue_Sawai Mansingh Stadium', 'venue_Shaheed Veer Narayan Singh International Stadium', 
         'venue_Sharjah Cricket Stadium', 'venue_Sheikh Zayed Stadium', 'venue_Subrata Roy Sahara Stadium', 'venue_Wankhede Stadium',]]

In [17]:
train = df[df['year'] <= 2016]
test = df[df['year'] > 2016]

In [18]:
X_train = train.drop(['year', 'total'], axis = 1)
X_test = test.drop(['year', 'total'], axis = 1)
y_train = train['total']
y_test = test['total']

In [19]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((29344, 49), (5341, 49), (29344,), (5341,))

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [21]:
y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)
print('Linear Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_lr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_lr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_lr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_lr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_lr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_lr) / y_test))

Linear Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.656623416184033
Test r2_score:  0.6162551780358305
----------------------------------------------------------------------------------------------------
Train RMSE:  16.811235290579244
Test RMSE:  18.208928038125112
----------------------------------------------------------------------------------------------------
Train MAPE:  0.08190542867396797
Test MAPE:  0.0890134624057405


In [22]:
data_lr = pd.DataFrame()
data_lr['actual'] = y_test
data_lr['predicted'] = y_test_pred_lr
data_lr['lower range'] = y_test_pred_lr - (0.0890134624057405 * y_test_pred_lr)
data_lr['upper range'] = y_test_pred_lr + (0.0890134624057405 * y_test_pred_lr)
t_list = []
for i in data_lr.index:
    if data_lr['actual'][i] > data_lr['lower range'][i]:
        if data_lr['actual'][i] < data_lr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_lr['Bool'] = t_list
data_lr['Bool'].value_counts()/data_lr.shape[0]

True     0.615053
False    0.384947
Name: Bool, dtype: float64

In [23]:
la = Lasso()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
la_rscv = RandomizedSearchCV(estimator = la, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
la_rscv.fit(X_train, y_train)
la_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] .

[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   10.8s finished


{'alpha': 0.314}

In [24]:
y_train_pred_la = la_rscv.predict(X_train)
y_test_pred_la = la_rscv.predict(X_test)
print('Lasso Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_la))
print('Test r2_score: ', r2_score(y_test, y_test_pred_la))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_la)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_la)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_la) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_la) / y_test))

Lasso Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6300002598739947
Test r2_score:  0.6663102068815934
----------------------------------------------------------------------------------------------------
Train RMSE:  17.450786222562808
Test RMSE:  16.97987843773432
----------------------------------------------------------------------------------------------------
Train MAPE:  0.0832273289172532
Test MAPE:  0.08037740543792475


In [25]:
data_la = pd.DataFrame()
data_la['actual'] = y_test
data_la['predicted'] = y_test_pred_la
data_la['lower range'] = y_test_pred_la - (0.08037740543792475 * y_test_pred_la)
data_la['upper range'] = y_test_pred_la + (0.08037740543792475 * y_test_pred_la)
t_list = []
for i in data_la.index:
    if data_la['actual'][i] > data_la['lower range'][i]:
        if data_la['actual'][i] < data_la['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_la['Bool'] = t_list
data_la['Bool'].value_counts()/data_la.shape[0]

True     0.625913
False    0.374087
Name: Bool, dtype: float64

In [26]:
ri = Ridge()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
ri_rscv = RandomizedSearchCV(estimator = ri, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
ri_rscv.fit(X_train, y_train)
ri_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.242 .....................................................
[CV] .

[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.0s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.0s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.0s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.0s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.0s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.5s finished


{'alpha': 0.942}

In [27]:
y_train_pred_ri = ri_rscv.predict(X_train)
y_test_pred_ri = ri_rscv.predict(X_test)
print('Ridge Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_ri))
print('Test r2_score: ', r2_score(y_test, y_test_pred_ri))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_ri)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_ri)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_ri) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_ri) / y_test))

Ridge Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6566208196126131
Test r2_score:  0.6164987910476469
----------------------------------------------------------------------------------------------------
Train RMSE:  16.81129885270407
Test RMSE:  18.203147326845887
----------------------------------------------------------------------------------------------------
Train MAPE:  0.08189333140534527
Test MAPE:  0.08898039302538752


In [28]:
data_ri = pd.DataFrame()
data_ri['actual'] = y_test
data_ri['predicted'] = y_test_pred_ri
data_ri['lower range'] = y_test_pred_ri - (0.08898039302538752 * y_test_pred_ri)
data_ri['upper range'] = y_test_pred_ri + (0.08898039302538752 * y_test_pred_ri)
t_list = []
for i in data_ri.index:
    if data_ri['actual'][i] > data_ri['lower range'][i]:
        if data_ri['actual'][i] < data_ri['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_ri['Bool'] = t_list
data_ri['Bool'].value_counts()/data_ri.shape[0]

True     0.615053
False    0.384947
Name: Bool, dtype: float64

In [29]:
dtr = DecisionTreeRegressor()
possible_parameter_values = {'criterion' : ['mse', 'friedman_mse', 'mae'],
                             'splitter' : ['best', 'random'],
                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)],
                             'min_samples_split' : [int(x) for x in np.arange(start = 5, stop = 101, step = 1)],
                             'min_samples_leaf' : [int(x) for x in np.arange(start = 1, stop = 51, step = 1)]}
dtr_rscv = RandomizedSearchCV(estimator = dtr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
dtr_rscv.fit(X_train, y_train)
dtr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.1

[CV]  splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse, total=   0.1

[CV]  splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae, total=  37.8s
[CV] splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae 
[CV]  splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae, total=  28.8s
[CV] splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae 
[CV]  splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae, total=  34.3s
[CV] splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae 
[CV]  splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae, total=  32.2s
[CV] splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae 
[CV]  splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae, total=  41.0s
[CV] splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, 

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 16.6min finished


{'splitter': 'best',
 'min_samples_split': 28,
 'min_samples_leaf': 10,
 'max_depth': 5,
 'criterion': 'mse'}

In [30]:
y_train_pred_dtr = dtr_rscv.predict(X_train)
y_test_pred_dtr = dtr_rscv.predict(X_test)
print('Decision Tree Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_dtr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_dtr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_dtr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_dtr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_dtr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_dtr) / y_test))

Decision Tree Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5433355536589235
Test r2_score:  0.5196537290662708
----------------------------------------------------------------------------------------------------
Train RMSE:  19.387101732200232
Test RMSE:  20.37231239367301
----------------------------------------------------------------------------------------------------
Train MAPE:  0.09596889752810353
Test MAPE:  0.09795377510230392


In [31]:
data_dtr = pd.DataFrame()
data_dtr['actual'] = y_test
data_dtr['predicted'] = y_test_pred_dtr
data_dtr['lower range'] = y_test_pred_dtr - (0.09795377510230392 * y_test_pred_dtr)
data_dtr['upper range'] = y_test_pred_dtr + (0.09795377510230392 * y_test_pred_dtr)
t_list = []
for i in data_dtr.index:
    if data_dtr['actual'][i] > data_dtr['lower range'][i]:
        if data_dtr['actual'][i] < data_dtr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_dtr['Bool'] = t_list
data_dtr['Bool'].value_counts()/data_dtr.shape[0]

True     0.621794
False    0.378206
Name: Bool, dtype: float64

In [32]:
rfr = RandomForestRegressor()
possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)],
                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)],
                             'min_samples_split' : [int(x) for x in np.arange(start = 5, stop = 101, step = 1)],
                             'min_samples_leaf' : [int(x) for x in np.arange(start = 1, stop = 51, step = 1)]}
rfr_rscv = RandomizedSearchCV(estimator = rfr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
rfr_rscv.fit(X_train, y_train)
rfr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=  14.0s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.9s remaining:    0.0s


[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=  13.2s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=  12.9s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=  13.0s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=  12.8s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=  12.6s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=  13.8s
[CV] n_estimators=200, min_sam

[CV]  n_estimators=500, min_samples_split=65, min_samples_leaf=32, max_depth=45, total=  29.6s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   3.0s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   2.8s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   2.7s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   2.6s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   2.7s
[CV] n_estimators=50, min_samples_split=

[CV]  n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20, total=  14.3s
[CV] n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20 
[CV]  n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20, total=  14.5s
[CV] n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20 
[CV]  n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20, total=  13.4s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 46.8min finished


{'n_estimators': 800,
 'min_samples_split': 89,
 'min_samples_leaf': 50,
 'max_depth': 15}

In [33]:
y_train_pred_rfr = rfr_rscv.predict(X_train)
y_test_pred_rfr = rfr_rscv.predict(X_test)
print('Random Forest Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_rfr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_rfr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_rfr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_rfr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_rfr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_rfr) / y_test))

Random Forest Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.7556185357578535
Test r2_score:  0.634343916893259
----------------------------------------------------------------------------------------------------
Train RMSE:  14.182357222328058
Test RMSE:  17.774586929405817
----------------------------------------------------------------------------------------------------
Train MAPE:  0.06805242582767848
Test MAPE:  0.08427261254533734


In [34]:
data_rfr = pd.DataFrame()
data_rfr['actual'] = y_test
data_rfr['predicted'] = y_test_pred_rfr
data_rfr['lower range'] = y_test_pred_rfr - (0.08427261254533734 * y_test_pred_rfr)
data_rfr['upper range'] = y_test_pred_rfr + (0.08427261254533734 * y_test_pred_rfr)
t_list = []
for i in data_rfr.index:
    if data_rfr['actual'][i] > data_rfr['lower range'][i]:
        if data_rfr['actual'][i] < data_rfr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_rfr['Bool'] = t_list
data_rfr['Bool'].value_counts()/data_rfr.shape[0]

True     0.619734
False    0.380266
Name: Bool, dtype: float64

In [35]:
abr = AdaBoostRegressor()
possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)],
                             'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
abr_rscv = RandomizedSearchCV(estimator = abr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
abr_rscv.fit(X_train, y_train)
abr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] n_estimators=200, learning_rate=0.544 ...........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ n_estimators=200, learning_rate=0.544, total=   2.1s
[CV] n_estimators=200, learning_rate=0.544 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.9s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.9s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.6s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   2.0s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   5.3s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.8s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   3.0s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] .

[CV] ............ n_estimators=250, learning_rate=0.229, total=   5.0s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   1.7s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   1.5s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   2.1s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   2.3s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   2.1s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   3.7s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 11.7min finished


{'n_estimators': 350, 'learning_rate': 0.021}

In [36]:
y_train_pred_abr = abr_rscv.predict(X_train)
y_test_pred_abr = abr_rscv.predict(X_test)
print('Ada Boost Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_abr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_abr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_abr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_abr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_abr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_abr) / y_test))

Ada Boost Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5099577332763432
Test r2_score:  0.48186611681369806
----------------------------------------------------------------------------------------------------
Train RMSE:  20.083114246033098
Test RMSE:  21.158462800565864
----------------------------------------------------------------------------------------------------
Train MAPE:  0.10132603379271818
Test MAPE:  0.10124573359584271


In [37]:
data_abr = pd.DataFrame()
data_abr['actual'] = y_test
data_abr['predicted'] = y_test_pred_abr
data_abr['lower range'] = y_test_pred_abr - (0.10124573359584271 * y_test_pred_abr)
data_abr['upper range'] = y_test_pred_abr + (0.10124573359584271 * y_test_pred_abr)
t_list = []
for i in data_abr.index:
    if data_abr['actual'][i] > data_abr['lower range'][i]:
        if data_abr['actual'][i] < data_abr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_abr['Bool'] = t_list
data_abr['Bool'].value_counts()/data_abr.shape[0]

True     0.607002
False    0.392998
Name: Bool, dtype: float64

In [38]:
knr = KNeighborsRegressor()
possible_parameter_values = {'n_neighbors' : [int(x) for x in np.arange(start = 1, stop = 11, step = 1)],
                             'weights' : ['uniform', 'distance']}
knr_rscv = RandomizedSearchCV(estimator = knr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
knr_rscv.fit(X_train, y_train)
knr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] weights=uniform, n_neighbors=8 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... weights=uniform, n_neighbors=8, total=   0.5s
[CV] weights=uniform, n_neighbors=8 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ................... weights=uniform, n_neighbors=8, total=   0.4s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.5s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.5s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.5s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.5s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.5s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.5s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] .

[CV] ................... weights=uniform, n_neighbors=5, total=   0.4s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.5s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.5s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.4s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.4s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.5s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.4s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   44.3s finished


{'weights': 'uniform', 'n_neighbors': 8}

In [39]:
y_train_pred_knr = knr_rscv.predict(X_train)
y_test_pred_knr = knr_rscv.predict(X_test)
print('K Neighbors Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_knr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_knr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_knr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_knr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_knr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_knr) / y_test))

K Neighbors Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.8260128118881142
Test r2_score:  0.5538009143266089
----------------------------------------------------------------------------------------------------
Train RMSE:  11.966659216042917
Test RMSE:  19.634843921340206
----------------------------------------------------------------------------------------------------
Train MAPE:  0.05328509596416751
Test MAPE:  0.09185259348772307


In [40]:
data_knr = pd.DataFrame()
data_knr['actual'] = y_test
data_knr['predicted'] = y_test_pred_knr
data_knr['lower range'] = y_test_pred_knr - (0.09185259348772307 * y_test_pred_knr)
data_knr['upper range'] = y_test_pred_knr + (0.09185259348772307 * y_test_pred_knr)
t_list = []
for i in data_knr.index:
    if data_knr['actual'][i] > data_knr['lower range'][i]:
        if data_knr['actual'][i] < data_knr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_knr['Bool'] = t_list
data_knr['Bool'].value_counts()/data_knr.shape[0]

True     0.636585
False    0.363415
Name: Bool, dtype: float64

In [41]:
xgbr = XGBRegressor()
possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)], 
                             'use_label_encoder' : [True, False], 
                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)], 
                             'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
xgbr_rscv = RandomizedSearchCV(estimator = xgbr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, 
                               random_state = 17, n_jobs = 1)
xgbr_rscv.fit(X_train, y_train)
xgbr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   5.8s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.7s remaining:    0.0s


[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   4.5s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   4.3s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   4.1s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   3.8s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   3.8s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   3.9s
[CV] use

[CV]  use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971, total=   5.0s
[CV] use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971 
[CV]  use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971, total=   4.9s
[CV] use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971 
[CV]  use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971, total=   5.2s
[CV] use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646 
[CV]  use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646, total=   5.5s
[CV] use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646 
[CV]  use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646, total=   5.6s
[CV] use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646 
[CV]  use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646, total=   5.6s
[CV

[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  43.8s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  43.2s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  45.4s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  47.1s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  46.7s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  47.2s
[CV] use_label_encoder=True, n

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 18.0min finished


{'use_label_encoder': True,
 'n_estimators': 50,
 'max_depth': 15,
 'learning_rate': 0.158}

In [42]:
y_train_pred_xgbr = xgbr_rscv.predict(X_train)
y_test_pred_xgbr = xgbr_rscv.predict(X_test)
print('XGBoost Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_xgbr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_xgbr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_xgbr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_xgbr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_xgbr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_xgbr) / y_test))

XGBoost Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.9954155926736095
Test r2_score:  0.49010348574639195
----------------------------------------------------------------------------------------------------
Train RMSE:  1.942477011321251
Test RMSE:  20.989598773394167
----------------------------------------------------------------------------------------------------
Train MAPE:  0.007696522138562351
Test MAPE:  0.09945668996057899


In [43]:
data_xgbr = pd.DataFrame()
data_xgbr['actual'] = y_test
data_xgbr['predicted'] = y_test_pred_xgbr
data_xgbr['lower range'] = y_test_pred_xgbr - (0.09945668996057899 * y_test_pred_xgbr)
data_xgbr['upper range'] = y_test_pred_xgbr + (0.09945668996057899 * y_test_pred_xgbr)
t_list = []
for i in data_xgbr.index:
    if data_xgbr['actual'][i] > data_xgbr['lower range'][i]:
        if data_xgbr['actual'][i] < data_xgbr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_xgbr['Bool'] = t_list
data_xgbr['Bool'].value_counts()/data_xgbr.shape[0]

True     0.600449
False    0.399551
Name: Bool, dtype: float64

In [ ]:
'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5', 'total', 'year', 'bat_team_Deccan Chargers', 'bat_team_Delhi Capitals', 'bat_team_Gujarat Lions', 
'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians', 'bat_team_Pune Warriors', 
'bat_team_Rajasthan Royals', 'bat_team_Rising Pune Supergiant', 'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad', 
'bowl_team_Deccan Chargers', 'bowl_team_Delhi Capitals', 'bowl_team_Gujarat Lions', 'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders', 
'bowl_team_Mumbai Indians', 'bowl_team_Pune Warriors', 'bowl_team_Rajasthan Royals', 'bowl_team_Rising Pune Supergiant', 'bowl_team_Royal Challengers Bangalore', 
'bowl_team_Sunrisers Hyderabad', 'venue_Brabourne Stadium', 'venue_Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', 
'venue_Dubai International Cricket Stadium', 'venue_Eden Gardens', 'venue_Feroz Shah Kotla', 'venue_Green Park', 
'venue_Himachal Pradesh Cricket Association Stadium', 'venue_Holkar Cricket Stadium', 'venue_JSCA International Stadium Complex', 
'venue_M Chinnaswamy Stadium', 'venue_MA Chidambaram Stadium, Chepauk', 'venue_Maharashtra Cricket Association Stadium', 
'venue_Punjab Cricket Association Stadium, Mohali', 'venue_Rajiv Gandhi International Stadium, Uppal', 'venue_Sardar Patel Stadium, Motera', 
'venue_Saurashtra Cricket Association Stadium', 'venue_Sawai Mansingh Stadium', 'venue_Shaheed Veer Narayan Singh International Stadium', 
'venue_Sharjah Cricket Stadium', 'venue_Sheikh Zayed Stadium', 'venue_Subrata Roy Sahara Stadium', 'venue_Wankhede Stadium'

In [49]:
arr = np.array([86, 1, 10, 56, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [50]:
arr = arr.reshape(1, -1)

In [51]:
s = knr_rscv.predict(arr)[0]
s

184.25

In [52]:
s - (0.09185259348772307 * s)

167.32615964988702

In [53]:
s + (0.09185259348772307 * s)

201.17384035011298